В этой практической тетрадке мы изучим различные методы нормализации текста – стемматизацию и лемматизацию – и сравним, как работают основные библиотеки обработки естественного языка для русского и английского языков.

Мы рассмотрим:

* Разницу между стемматизацией и лемматизацией
* Сравнение работы библиотек NLTK, spaCy, PyMorphy2 и Natasha
* Создание комплексных пайплайнов препроцессинга для разных языков
* Практические упражнения для закрепления материала

In [ ]:
# Установка необходимых библиотек
!pip install nltk spacy pymorphy2 natasha
!python -m spacy download en_core_web_sm
!python -m spacy download ru_core_news_sm

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import spacy
import pymorphy2
from natasha import Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, Doc

  Using cached nltk-3.9.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached spacy-3.8.7-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached pymorphy2-0.9.1-py3-none-any.whl.metadata (3.6 kB)
  Using cached natasha-1.6.0-py3-none-any.whl.metadata (23 kB)
  Using cached click-8.3.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached regex-2025.9.18-cp311-cp311-win_amd64.whl.metadata (41 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached murmurhash-1.0.13-cp311-cp311-win_amd64.whl.metadata (2.2 kB)
  Using cached cymem-2.0.11-cp311-cp311-win_amd64.whl.metadata (8.8 kB)
  Using cached preshed-3.0.10-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
  Using cached thinc-8.3.6-cp311-cp311-win_amd64.whl.metadata (15 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 217.9 kB/s eta 0:00:59
     --------------------------------------- 0.0/12.8 MB 186.2 kB/s eta 0:01:09
     --------------------------------------- 0.0/12.8 MB 186.2 kB/s eta 0:01:09
     --------------------------------------- 0.1/12.8 MB 217.9 kB/s eta 0:00:59
     --------------------------------------- 0.1/12.8 MB 233.8 kB/s eta 0:00:55
     --------------------------------------- 0.1/12.8 MB 291.5 kB/s eta 0:00:44
     --------------------------------------- 0.1/12.8 MB 310.3 kB/s eta 0:00:41
     --------------------------------------- 0.1/12.8 MB 327.7 kB/s eta 0:00:39
      -------------------------------------- 0.2/12.8 MB 412.5 kB/s eta 0:00:31
      -------------------------------------- 0.2/12.8 MB 467.6 kB/s eta 0:00:27
      -------------------------------------- 0.3/12.8 MB


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     --------------------------------------- 0.0/15.3 MB 165.2 kB/s eta 0:01:33
     --------------------------------------- 0.0/15.3 MB 187.9 kB/s eta 0:01:22
     --------------------------------------- 0.0/15.3 MB 187.9 kB/s eta 0:01:22
     --------------------------------------- 0.0/15.3 MB 179.6 kB/s eta 0:01:25
     --------------------------------------- 0.1/15.3 MB 218.8 kB/s eta 0:01:10
     --------------------------------------- 0.1/15.3 MB 231.8 kB/s eta 0:01:06
     --------------------------------------- 0.1/15.3 MB 343.4 kB/s eta 0:00:45
     --------------------------------------- 0.1/15.3 MB 355.0 kB/s eta 0:00:43
     --------------------------------------- 0.2/15.3 MB 454.0 kB/s eta 0:00:34
      -------------------------------------- 0.2/15.3 MB 492.1 kB/s eta 0:00:31
      -------------------------------------- 0.3/15.3 MB


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...


**1. Сравнение стемматизации и лемматизации**

In [ ]:
# Тестовые тексты
russian_text = """
В России проживают народы с разными традициями и верованиями.
Русские люди всегда с интересом относились к культуре соседних народов.
Петр Первый ввел многие европейские обычаи.
Екатерина Великая переписывалась с французскими просветителями.
Современные россияне любят путешествовать по разным странам мира.
"""

english_text = """
The United States is home to people with diverse traditions and beliefs.
Americans have always been interested in the cultures of neighboring countries.
George Washington established many governmental customs.
Thomas Jefferson corresponded with French intellectuals.
Modern Americans enjoy traveling to different countries around the world.
"""

1.1 Стемматизация с помощью NLTK. Стемматизация - это процесс нахождения основы слова путем отбрасывания аффиксов (окончаний, суффиксов). Стемматизаторы используют набор правил без словарей.




In [ ]:
from nltk.stem import SnowballStemmer
import re

# Создаем стемматизаторы для английского и русского языков
english_stemmer = SnowballStemmer("english")
russian_stemmer = SnowballStemmer("russian")

# Функция для токенизации и стемматизации
def stem_text(text, stemmer):
    # Приводим к нижнему регистру и токенизируем
    words = re.findall(r'\b\w+\b', text.lower())
    # Применяем стемматизацию
    stemmed_words = [stemmer.stem(word) for word in words]
    return stemmed_words

# Стемматизация русского текста
russian_stemmed = stem_text(russian_text, russian_stemmer)
print("Стемматизированные русские слова:")
for i, word in enumerate(russian_stemmed[:15]):  # Выводим первые 15 слов
    print(f"{i+1}. {word}")

# Стемматизация английского текста
english_stemmed = stem_text(english_text, english_stemmer)
print("\nСтемматизированные английские слова:")
for i, word in enumerate(english_stemmed[:15]):  # Выводим первые 15 слов
    print(f"{i+1}. {word}")

Стемматизированные русские слова:
1. в
2. росс
3. прожива
4. народ
5. с
6. разн
7. традиц
8. и
9. верован
10. русск
11. люд
12. всегд
13. с
14. интерес
15. относ

Стемматизированные английские слова:
1. the
2. unit
3. state
4. is
5. home
6. to
7. peopl
8. with
9. divers
10. tradit
11. and
12. belief
13. american
14. have
15. alway


1.2 Лемматизация с помощью разных библиотек. Лемматизация - это процесс приведения слова к его словарной (нормальной) форме. Лемматизаторы используют словари и учитывают грамматические особенности.

1.2.1 NLTK (только для английского)

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.


True

In [ ]:
# Инициализация лемматизатора
lemmatizer = WordNetLemmatizer()

# Функция для конвертации POS-тегов NLTK в формат WordNet
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # По умолчанию существительное

# Функция для лемматизации с учетом части речи
def lemmatize_with_pos(text):
    # Токенизация и определение части речи
    tokens = word_tokenize(text.lower())
    tagged = pos_tag(tokens)

    # Лемматизация с учетом части речи
    lemmas = []
    for word, tag in tagged:
        if word.isalpha():  # Исключаем числа и знаки препинания
            wordnet_pos = get_wordnet_pos(tag)
            lemmas.append(lemmatizer.lemmatize(word, wordnet_pos))

    return lemmas

# Лемматизация английского текста
english_lemmas_nltk = lemmatize_with_pos(english_text)
print("Лемматизированные английские слова (NLTK):")
for i, word in enumerate(english_lemmas_nltk[:15]):
    print(f"{i+1}. {word}")

# Примечание: NLTK не имеет встроенного лемматизатора для русского языка
print("\nNLTK не поддерживает лемматизацию русского языка напрямую.")

Лемматизированные английские слова (NLTK):
1. the
2. united
3. state
4. be
5. home
6. to
7. people
8. with
9. diverse
10. tradition
11. and
12. belief
13. american
14. have
15. always

NLTK не поддерживает лемматизацию русского языка напрямую.


1.2.2 spaCy (для обоих языков)

In [ ]:
# Загрузка моделей spaCy
nlp_en = spacy.load("en_core_web_sm")
nlp_ru = spacy.load("ru_core_news_sm")

# Функция для лемматизации с помощью spaCy
def lemmatize_with_spacy(text, nlp):
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc if token.is_alpha]
    return lemmas

# Лемматизация английского текста
english_lemmas_spacy = lemmatize_with_spacy(english_text, nlp_en)
print("Лемматизированные английские слова (spaCy):")
for i, word in enumerate(english_lemmas_spacy[:15]):
    print(f"{i+1}. {word}")

# Лемматизация русского текста
russian_lemmas_spacy = lemmatize_with_spacy(russian_text, nlp_ru)
print("\nЛемматизированные русские слова (spaCy):")
for i, word in enumerate(russian_lemmas_spacy[:15]):
    print(f"{i+1}. {word}")

Лемматизированные английские слова (spaCy):
1. the
2. United
3. States
4. be
5. home
6. to
7. people
8. with
9. diverse
10. tradition
11. and
12. belief
13. Americans
14. have
15. always

Лемматизированные русские слова (spaCy):
1. в
2. россия
3. проживать
4. народ
5. с
6. разный
7. традиция
8. и
9. верование
10. русский
11. человек
12. всегда
13. с
14. интерес
15. относиться


1.2.3 PyMorphy2 (для русского языка)

In [ ]:
# Инициализация анализатора
morph = pymorphy2.MorphAnalyzer()

# Функция для лемматизации с помощью PyMorphy2
def lemmatize_with_pymorphy(text):
    # Токенизация
    words = re.findall(r'\b\w+\b', text.lower())

    # Лемматизация
    lemmas = [morph.parse(word)[0].normal_form for word in words]
    return lemmas

# Лемматизация русского текста
russian_lemmas_pymorphy = lemmatize_with_pymorphy(russian_text)
print("Лемматизированные русские слова (PyMorphy2):")
for i, word in enumerate(russian_lemmas_pymorphy[:15]):
    print(f"{i+1}. {word}")

Лемматизированные русские слова (PyMorphy2):
1. в
2. россия
3. проживать
4. народ
5. с
6. разный
7. традиция
8. и
9. верование
10. русский
11. человек
12. всегда
13. с
14. интерес
15. относиться


1.2.4 Natasha (для русского языка)

In [ ]:
# Инициализация компонентов Natasha
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

# Функция для лемматизации с помощью Natasha
def lemmatize_with_natasha(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    lemmas = []
    for token in doc.tokens:
        if token.text.isalpha():  # Исключаем числа и знаки препинания
            token.lemmatize(morph_vocab)
            lemmas.append(token.lemma)

    return lemmas

# Лемматизация русского текста
russian_lemmas_natasha = lemmatize_with_natasha(russian_text)
print("Лемматизированные русские слова (Natasha):")
for i, word in enumerate(russian_lemmas_natasha[:15]):
    print(f"{i+1}. {word}")

Лемматизированные русские слова (Natasha):
1. в
2. россия
3. проживать
4. народ
5. с
6. разный
7. традиция
8. и
9. верование
10. русский
11. человек
12. всегда
13. с
14. интерес
15. относиться


1.3 Сравнительная таблица результатов

Для наглядности сравним результаты обработки одних и тех же слов разными методами.

In [ ]:
import pandas as pd

# Выбираем несколько интересных слов из русского текста для сравнения
russian_words = ["России", "проживают", "народы", "традициями", "верованиями",
                "русские", "людей", "относились", "культуре", "Петр", "ввел", "обычаи"]

# Создаем DataFrame для сравнения
comparison_ru = []
for word in russian_words:
    stem = russian_stemmer.stem(word.lower())
    lemma_spacy = None

    # Находим лемму в spaCy
    doc = nlp_ru(word)
    for token in doc:
        lemma_spacy = token.lemma_

    lemma_pymorphy = morph.parse(word.lower())[0].normal_form

    # Находим лемму в Natasha
    doc = Doc(word)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        lemma_natasha = token.lemma

    comparison_ru.append({
        "Слово": word,
        "Стем (NLTK)": stem,
        "Лемма (spaCy)": lemma_spacy,
        "Лемма (PyMorphy2)": lemma_pymorphy,
        "Лемма (Natasha)": lemma_natasha
    })

# Создаем таблицу для русских слов
comparison_ru_df = pd.DataFrame(comparison_ru)
print("Сравнение методов для русского языка:")
comparison_ru_df

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Выбираем несколько интересных слов из английского текста для сравнения
english_words = ["States", "people", "diverse", "traditions", "beliefs",
                "Americans", "interested", "cultures", "neighboring", "established", "corresponded", "traveling"]

# Создаем DataFrame для сравнения
comparison_en = []
for word in english_words:
    stem = english_stemmer.stem(word.lower())

    # Находим лемму в NLTK
    pos = pos_tag([word.lower()])[0][1]
    wordnet_pos = get_wordnet_pos(pos)
    lemma_nltk = lemmatizer.lemmatize(word.lower(), wordnet_pos)

    # Находим лемму в spaCy
    doc = nlp_en(word)
    for token in doc:
        lemma_spacy = token.lemma_

    comparison_en.append({
        "Слово": word,
        "Стем (NLTK)": stem,
        "Лемма (NLTK)": lemma_nltk,
        "Лемма (spaCy)": lemma_spacy
    })

# Создаем таблицу для английских слов
comparison_en_df = pd.DataFrame(comparison_en)
print("Сравнение методов для английского языка:")
comparison_en_df

NameError: name 'pd' is not defined

**2. Комплексные пайплайны препроцессинга текста**

2.1 Пайплайн для русского языка с PyMorphy2

In [ ]:
import re
import pymorphy2
from nltk.corpus import stopwords

In [ ]:
# Загрузка стоп-слов
nltk.download('stopwords')
russian_stopwords = set(stopwords.words('russian'))

# Инициализация анализатора
morph = pymorphy2.MorphAnalyzer()

def preprocess_russian_text(text):
    """
    Полный пайплайн предобработки русского текста с использованием PyMorphy2
    """
    # Шаг 1: Приведение текста к нижнему регистру
    text = text.lower()
    print(f"1. Приведение к нижнему регистру: {text[:50]}...")

    # Шаг 2: Замена 'ё' на 'е'
    text = text.replace('ё', 'е')
    print(f"2. Замена 'ё' на 'е': {text[:50]}...")

    # Шаг 3: Удаление цифр, знаков препинания и лишних пробелов
    text = re.sub(r'[^\w\s]', ' ', text)  # Удаление знаков препинания
    text = re.sub(r'\d+', ' ', text)      # Удаление цифр
    text = re.sub(r'\s+', ' ', text)      # Удаление лишних пробелов
    text = text.strip()                    # Удаление пробелов в начале и конце
    print(f"3. Очистка текста: {text[:50]}...")

    # Шаг 4: Токенизация
    tokens = text.split()
    print(f"4. Токенизация: получено {len(tokens)} токенов")

    # Шаг 5: Удаление стоп-слов
    filtered_tokens = [token for token in tokens if token not in russian_stopwords]
    print(f"5. Удаление стоп-слов: осталось {len(filtered_tokens)} токенов")

    # Шаг 6: Лемматизация с PyMorphy2
    lemmas = []
    for token in filtered_tokens:
        parsed = morph.parse(token)[0]
        lemmas.append(parsed.normal_form)

    print(f"6. Лемматизация: получено {len(lemmas)} лемм")

    # Шаг 7: Удаление слишком коротких слов
    final_lemmas = [lemma for lemma in lemmas if len(lemma) > 2]
    print(f"7. Фильтрация коротких слов: итоговое количество {len(final_lemmas)} лемм")

    return final_lemmas

# Применение пайплайна к русскому тексту
print("Комплексный пайплайн для русского текста:")
preprocessed_russian = preprocess_russian_text(russian_text)
print("\nРезультат:")
print(preprocessed_russian)

Комплексный пайплайн для русского текста:
1. Приведение к нижнему регистру: 
в россии проживают народы с разными традициями и ...
2. Замена 'ё' на 'е': 
в россии проживают народы с разными традициями и ...
3. Очистка текста: в россии проживают народы с разными традициями и в...
4. Токенизация: получено 39 токенов
5. Удаление стоп-слов: осталось 31 токенов
6. Лемматизация: получено 31 лемм
7. Фильтрация коротких слов: итоговое количество 31 лемм

Результат:
['россия', 'проживать', 'народ', 'разный', 'традиция', 'верование', 'русский', 'человек', 'интерес', 'относиться', 'культура', 'соседний', 'народ', 'пётр', 'первый', 'ввести', 'многие', 'европейский', 'обычай', 'екатерина', 'великий', 'переписываться', 'французский', 'просветитель', 'современный', 'россиянин', 'любить', 'путешествовать', 'разный', 'страна', 'мир']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2.2 Пайплайн для английского языка с spaCy

In [ ]:
import spacy
from nltk.corpus import stopwords

In [ ]:
# Загрузка стоп-слов и модели spaCy
english_stopwords = set(stopwords.words('english'))
nlp_en = spacy.load("en_core_web_sm")

def preprocess_english_text(text):
    """
    Полный пайплайн предобработки английского текста с использованием spaCy
    """
    # Шаг 1: Приведение текста к нижнему регистру
    text = text.lower()
    print(f"1. Приведение к нижнему регистру: {text[:50]}...")

    # Шаг 2: Расширение сокращений (контракций)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"'ve", " have", text)
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"'m", " am", text)
    text = re.sub(r"'ll", " will", text)
    text = re.sub(r"'d", " would", text)
    print(f"2. Расширение контракций: {text[:50]}...")

    # Шаг 3: Обработка текста с помощью spaCy
    doc = nlp_en(text)
    print(f"3. Обработка с spaCy: получено {len(doc)} токенов")

    # Шаг 4: Фильтрация и лемматизация
    lemmas = []
    for token in doc:
        # Проверяем, что токен не является стоп-словом, пунктуацией или числом
        if (not token.is_stop and
            not token.is_punct and
            not token.is_digit and
            token.is_alpha):  # Только буквенные токены

            # Получаем лемму и проверяем, что она не является специальным токеном spaCy
            lemma = token.lemma_
            if lemma != '-PRON-':  # spaCy иногда использует -PRON- для местоимений
                lemmas.append(lemma)

    print(f"4. Фильтрация и лемматизация: получено {len(lemmas)} лемм")

    # Шаг 5: Удаление слишком коротких слов
    final_lemmas = [lemma for lemma in lemmas if len(lemma) > 2]
    print(f"5. Фильтрация коротких слов: итоговое количество {len(final_lemmas)} лемм")

    return final_lemmas

# Применение пайплайна к английскому тексту
print("Комплексный пайплайн для английского текста:")
preprocessed_english = preprocess_english_text(english_text)
print("\nРезультат:")
print(preprocessed_english)

Комплексный пайплайн для английского текста:
1. Приведение к нижнему регистру: 
the united states is home to people with diverse ...
2. Расширение контракций: 
the united states is home to people with diverse ...
3. Обработка с spaCy: получено 56 токенов
4. Фильтрация и лемматизация: получено 29 лемм
5. Фильтрация коротких слов: итоговое количество 29 лемм

Результат:
['united', 'states', 'home', 'people', 'diverse', 'tradition', 'belief', 'americans', 'interested', 'culture', 'neighboring', 'country', 'george', 'washington', 'establish', 'governmental', 'custom', 'thomas', 'jefferson', 'correspond', 'french', 'intellectual', 'modern', 'americans', 'enjoy', 'travel', 'different', 'country', 'world']


2.3 Пайплайн для смешанного текста (русский + английский)

Блок 1: Импорт необходимых библиотек и инициализация

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import pymorphy2

# Загрузим необходимые ресурсы
nltk.download('stopwords')
nltk.download('punkt')

# Инициализация анализаторов
morph = pymorphy2.MorphAnalyzer()
russian_stemmer = SnowballStemmer("russian")
english_stemmer = SnowballStemmer("english")

# Списки стоп-слов
russian_stop_words = set(stopwords.words('russian'))
english_stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Исходный текст
mixed_text = """
<p>В современном мире digital-технологии стали неотъемлемой частью жизни.</p>
<strong>Information technologies</strong> изменили способы коммуникации между людьми в 2023 году.
CEO крупных IT-компаний (более 500 сотрудников) регулярно проводят online-конференции.
<a href="https://example.com">Developers</a> из разных стран сотрудничают в open-source проектах.
Искусственный интеллект и <em>machine learning</em> находят применение в разных сферах.
Для связи: contact@tech-info.com или посетите https://tech-conference.org.
<div class="footer">
  © 2023 AI-News. Телефон: +7 (123) 456-78-90. #искусственныйинтеллект #bigdata
</div>
Data-scientists и ML-инженеры работают с web-приложениями и AI-моделями.
Кибер-безопасность и e-commerce — важные направления IT-индустрии.
"""

Блок 2: Нормализация текста

In [ ]:
def normalize_mixed_text(text):
    """
    Нормализует смешанный текст, удаляя HTML-теги, URL, email, специальные символы
    и выполняя базовые преобразования для стандартизации текста.

    Args:
        text (str): Исходный смешанный текст с "шумом"

    Returns:
        str: Нормализованный текст
    """
    # Шаг 1: Удаление HTML-тегов
    # Находит и удаляет всё между < и >, включая скрипты и стили
    text = re.sub(r'<[^>]+>', ' ', text)

    # Шаг 2: Удаление URL-адресов
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)

    # Шаг 3: Удаление email-адресов
    text = re.sub(r'\S+@\S+\.\S+', ' ', text)

    # Шаг 4: Удаление хештегов
    text = re.sub(r'#\S+', ' ', text)

    # Шаг 5: Удаление номеров телефонов (различные форматы)
    text = re.sub(r'\+\d+\s*[\(\)]*\s*\d+[\s\-\(\)]*\d+[\s\-\(\)]*\d+', ' ', text)

    # Шаг 6: Удаление символов копирайта, специальных символов
    text = re.sub(r'[©®™℠]+', ' ', text)

    # Шаг 7: Замена скобок и их содержимого на пробел
    text = re.sub(r'\([^)]*\)', ' ', text)

    # Шаг 8: Приведение к нижнему регистру
    text = text.lower()

    # Шаг 9: Замена буквы "ё" на "е" для унификации русского текста
    text = text.replace('ё', 'е')

    # Шаг 10: Обработка пунктуации
    # Сохраняем дефисы внутри слов, но удаляем остальную пунктуацию
    # Сначала заменим дефисы временным маркером
    text = re.sub(r'(\w)-(\w)', r'\1HYPHEN\2', text)

    # Удаляем пунктуацию кроме временных маркеров дефисов
    text = re.sub(r'[^\w\sHYPHEN]', ' ', text)

    # Возвращаем дефисы
    text = re.sub(r'HYPHEN', '-', text)

    # Шаг 11: Удаление цифр, но сохранение слов с цифрами (например, ML-инженеры5)
    text = re.sub(r'\b\d+\b', ' ', text)

    # Шаг 12: Удаление избыточных пробелов (в том числе в начале и конце строк)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Тест функции
normalized_text = normalize_mixed_text(mixed_text)
print("Нормализованный текст:")
print(normalized_text)

Нормализованный текст:
в современном мире digital-технологии стали неотъемлемой частью жизни information technologies изменили способы коммуникации между людьми в году ceo крупных it-компаний регулярно проводят online-конференции developers из разных стран сотрудничают в open-source проектах искусственный интеллект и machine learning находят применение в разных сферах для связи или посетите ai-news телефон data-scientists и ml-инженеры работают с web-приложениями и ai-моделями кибер-безопасность и e-commerce важные направления it-индустрии


Блок 3: Определение алфавита символа и токенизация

In [ ]:
def is_cyrillic(char):
    """Проверяет, является ли символ кириллическим."""
    return 'а' <= char <= 'я' or char == 'ё'

def is_latin(char):
    """Проверяет, является ли символ латинским."""
    return 'a' <= char <= 'z'

def classify_token(token):
    """Определяет язык токена: 'russian', 'english' или 'mixed'."""
    if not token:
        return None

    # Удаляем все не-буквенные символы для определения языка
    letters = [c for c in token.lower() if c.isalpha()]
    if not letters:
        return None

    cyrillic_count = sum(1 for c in letters if is_cyrillic(c))
    latin_count = sum(1 for c in letters if is_latin(c))

    if cyrillic_count > 0 and latin_count == 0:
        return 'russian'
    elif latin_count > 0 and cyrillic_count == 0:
        return 'english'
    elif cyrillic_count > 0 and latin_count > 0:
        return 'mixed'
    else:
        return None

def split_hyphenated_words(token):
    """Разбивает слова с дефисом, если они состоят из смешанных символов."""
    if '-' not in token:
        return [token]

    # Проверяем, есть ли смешанные символы в токене
    if classify_token(token) != 'mixed':
        return [token]

    # Разбиваем по дефису
    parts = token.split('-')
    result = []

    for part in parts:
        if part:  # Игнорируем пустые части
            result.append(part)

    return result

def tokenize_and_classify(text):
    """Токенизирует текст и классифицирует токены по языку."""
    # Сначала разделим текст на слова, сохраняя дефисы внутри слов
    raw_tokens = re.findall(r'\b[\w\-]+\b', text)

    russian_tokens = []
    english_tokens = []

    for token in raw_tokens:
        # Проверяем, нужно ли разделить слово с дефисом
        split_tokens = split_hyphenated_words(token)

        for t in split_tokens:
            lang = classify_token(t)
            if lang == 'russian':
                russian_tokens.append(t)
            elif lang == 'english':
                english_tokens.append(t)

    return russian_tokens, english_tokens

# Применяем токенизацию и классификацию
russian_tokens, english_tokens = tokenize_and_classify(normalized_text)

print("\nРусские токены:")
print(russian_tokens)
print("\nАнглийские токены:")
print(english_tokens)


Русские токены:
['в', 'современном', 'мире', 'технологии', 'стали', 'неотъемлемой', 'частью', 'жизни', 'изменили', 'способы', 'коммуникации', 'между', 'людьми', 'в', 'году', 'крупных', 'компаний', 'регулярно', 'проводят', 'конференции', 'из', 'разных', 'стран', 'сотрудничают', 'в', 'проектах', 'искусственный', 'интеллект', 'и', 'находят', 'применение', 'в', 'разных', 'сферах', 'для', 'связи', 'или', 'посетите', 'телефон', 'и', 'инженеры', 'работают', 'с', 'приложениями', 'и', 'моделями', 'кибер-безопасность', 'и', 'важные', 'направления', 'индустрии']

Английские токены:
['digital', 'information', 'technologies', 'ceo', 'it', 'online', 'developers', 'open-source', 'machine', 'learning', 'ai-news', 'data-scientists', 'ml', 'web', 'ai', 'e-commerce', 'it']


Блок 4: Обработка токенов по языковым правилам

In [ ]:
def process_russian_tokens(tokens):
    """Обрабатывает русские токены по пайплайну для русского языка."""
    # Фильтрация стоп-слов
    filtered_tokens = [token for token in tokens if token not in russian_stop_words]

    # Лемматизация с помощью pymorphy2
    lemmatized_tokens = []
    for token in filtered_tokens:
        # Получаем нормальную форму
        lemma = morph.parse(token)[0].normal_form
        lemmatized_tokens.append(lemma)

    return lemmatized_tokens

def process_english_tokens(tokens):
    """Обрабатывает английские токены по пайплайну для английского языка."""
    # Обработка сокращений и апострофов
    expanded_tokens = []
    for token in tokens:
        # Обработка притяжательных форм и сокращений
        token = re.sub(r'\'s$|\'$', '', token)  # Удаляем 's и '
        expanded_tokens.append(token)

    # Фильтрация стоп-слов
    filtered_tokens = [token for token in expanded_tokens if token not in english_stop_words]

    # Стемматизация
    stemmed_tokens = [english_stemmer.stem(token) for token in filtered_tokens]

    return stemmed_tokens

# Обработка токенов по языковым правилам
processed_russian = process_russian_tokens(russian_tokens)
processed_english = process_english_tokens(english_tokens)

print("\nОбработанные русские токены:")
print(processed_russian)
print("\nОбработанные английские токены:")
print(processed_english)


Обработанные русские токены:
['современный', 'мир', 'технология', 'стать', 'неотъемлемый', 'часть', 'жизнь', 'изменить', 'способ', 'коммуникация', 'человек', 'год', 'крупный', 'компания', 'регулярно', 'проводить', 'конференция', 'разный', 'страна', 'сотрудничать', 'проект', 'искусственный', 'интеллект', 'находить', 'применение', 'разный', 'сфера', 'связь', 'посетить', 'телефон', 'инженер', 'работать', 'приложение', 'модель', 'кибер-безопасность', 'важный', 'направление', 'индустрия']

Обработанные английские токены:
['digit', 'inform', 'technolog', 'ceo', 'onlin', 'develop', 'open-sourc', 'machin', 'learn', 'ai-new', 'data-scientist', 'ml', 'web', 'ai', 'e-commerc']


Блок 5: Финальная обработка и объединение результатов

In [ ]:
def final_processing(russian_tokens, english_tokens):
    """Выполняет постобработку и объединяет результаты."""
    # Фильтрация коротких слов (менее 3 символов)
    filtered_russian = [token for token in russian_tokens if len(token) >= 3]
    filtered_english = [token for token in english_tokens if len(token) >= 3]

    # Удаление дубликатов
    unique_russian = list(dict.fromkeys(filtered_russian))
    unique_english = list(dict.fromkeys(filtered_english))

    return {
        'russian_tokens': unique_russian,
        'english_tokens': unique_english,
        'stats': {
            'russian_count': len(unique_russian),
            'english_count': len(unique_english),
            'total_unique': len(unique_russian) + len(unique_english)
        }
    }

# Финальная обработка
result = final_processing(processed_russian, processed_english)

print("\nФинальный результат:")
print("Русские слова:", result['russian_tokens'])
print("Английские слова:", result['english_tokens'])
print("\nСтатистика:")
print(f"Количество русских слов: {result['stats']['russian_count']}")
print(f"Количество английских слов: {result['stats']['english_count']}")
print(f"Общее количество уникальных слов: {result['stats']['total_unique']}")


Финальный результат:
Русские слова: ['современный', 'мир', 'технология', 'стать', 'неотъемлемый', 'часть', 'жизнь', 'изменить', 'способ', 'коммуникация', 'человек', 'год', 'крупный', 'компания', 'регулярно', 'проводить', 'конференция', 'разный', 'страна', 'сотрудничать', 'проект', 'искусственный', 'интеллект', 'находить', 'применение', 'сфера', 'связь', 'посетить', 'телефон', 'инженер', 'работать', 'приложение', 'модель', 'кибер-безопасность', 'важный', 'направление', 'индустрия']
Английские слова: ['digit', 'inform', 'technolog', 'ceo', 'onlin', 'develop', 'open-sourc', 'machin', 'learn', 'ai-new', 'data-scientist', 'web', 'e-commerc']

Статистика:
Количество русских слов: 37
Количество английских слов: 13
Общее количество уникальных слов: 50


**Практические упражнения:**

Упражнение 1: Базовая стемматизация русского текста с помощью NLTK

Задание:
Напишите функцию, которая принимает русский текст, выполняет токенизацию и применяет стемматизацию с использованием SnowballStemmer из библиотеки NLTK. Функция должна возвращать список стемматизированных слов.

Требования:

* Используйте SnowballStemmer("russian") из NLTK
* Удалите все знаки препинания перед стемматизацией
* Приведите слова к нижнему регистру
* Выведите исходные слова и их стеммы в виде таблицы для сравнения

In [22]:
import pandas as pd
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

russian_text = """
Программирование - это искусство создания программ для компьютеров.
Программисты разрабатывают различные приложения, которые используются миллионами людей.
Хорошие разработчики всегда учатся новому и совершенствуют свои навыки программирования.
"""

russian_stemmer = SnowballStemmer("russian")
tokens = word_tokenize(russian_text.lower())

words = [token for token in tokens if token.isalpha()]

data = []
for word in words:
    stem = russian_stemmer.stem(word)
    data.append([word, stem])

df = pd.DataFrame(data, columns =["Слово", "Стемма"])
print(df)

               Слово          Стемма
0   программирование  программирован
1                это              эт
2          искусство        искусств
3           создания          создан
4           программ        программ
5                для             для
6        компьютеров       компьютер
7       программисты     программист
8      разрабатывают     разрабатыва
9          различные         различн
10        приложения        приложен
11           которые           котор
12      используются         использ
13        миллионами         миллион
14             людей             люд
15           хорошие           хорош
16      разработчики     разработчик
17            всегда           всегд
18            учатся            учат
19            новому             нов
20                 и               и
21    совершенствуют     совершенств
22              свои             сво
23            навыки           навык
24  программирования  программирован


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Упражнение 2: Сравнение стемматизации и лемматизации английского текста

Задание: Реализуйте две функции, которые обрабатывают один и тот же английский текст с использованием PorterStemmer и WordNetLemmatizer из NLTK соответственно. Сравните результаты и определите, какой метод лучше сохраняет семантику слов.

Требования:

* Для стемматизации используйте PorterStemmer из NLTK
* Для лемматизации используйте WordNetLemmatizer с определением части речи
* Создайте таблицу сравнения из трех колонок: исходное слово, стемма, лемма
* Выделите случаи, где результаты стемматизации и лемматизации существенно различаются

In [21]:
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk import pos_tag

import pandas as pd

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')


english_text = """
Natural language processing helps computers understand and generate human language.
Researchers are developing better models that can recognize patterns in texts.
The running applications are processing huge amounts of data daily.
Many universities are teaching students how to build better language models.
"""

def get_wordnet_pos(tag):
    if tag.startswith("J"):
        return wordnet.ADJ
    elif tag.startswith("V"):
        return wordnet.VERB
    elif tag.startswith("N"):
        return wordnet.NOUN
    elif tag.startswith("R"):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
tokens = word_tokenize(english_text)
pos_tags = pos_tag(tokens)
    
data = []
for word, pos in pos_tags:
    stem = stemmer.stem(word)
    lemma = lemmatizer.lemmatize(word, get_wordnet_pos(pos))
    data.append([word, stem, lemma])

df = pd.DataFrame(data, columns=["Слово", "Стемма", "Лемма"])
df_diff = df[df["Стемма"] !=df["Лемма"]]

print("Таблица сравнения:")
print(df)
print("\nРазличия стемматизации и лемматизации:")
print(df_diff)

Таблица сравнения:
           Слово      Стемма        Лемма
0        Natural       natur      Natural
1       language     languag     language
2     processing     process   processing
3          helps        help         help
4      computers      comput     computer
5     understand  understand   understand
6            and         and          and
7       generate       gener     generate
8          human       human        human
9       language     languag     language
10             .           .            .
11   Researchers    research  Researchers
12           are         are           be
13    developing     develop      develop
14        better      better         good
15        models       model        model
16          that        that         that
17           can         can          can
18     recognize      recogn    recognize
19      patterns     pattern      pattern
20            in          in           in
21         texts        text         text
22             

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Упражнение 3: Лемматизация русского текста с использованием pymorphy2 и определение частей речи

Задание: Напишите функцию, которая выполняет морфологический анализ русского текста с помощью библиотеки pymorphy2. Функция должна возвращать для каждого слова его лемму (нормальную форму) и часть речи.

Требования:

* Используйте pymorphy2.MorphAnalyzer()
* Обработайте токенизированный текст, игнорируя пунктуацию и числа
* Для каждого слова определите нормальную форму и часть речи
* Посчитайте частоту встречаемости каждой части речи в тексте
* Найдите слова, имеющие омонимичные разборы, и предложите способ разрешения неоднозначности

In [20]:
import pymorphy2
import re
from collections import Counter

morph = pymorphy2.MorphAnalyzer()

def analyze_russian_text(text):
    words = re.findall(r'\b\w+\b', text.lower())
    
    results = []
    pos_counter = Counter()
    ambiguous_words = []
    
    for word in words:
        parsed = morph.parse(word)[0]
        
        lemma = parsed.normal_form
        pos = parsed.tag.POS
        
        pos_russian = {
            'NOUN': 'СУЩ', 'VERB': 'ГЛАГ', 'ADJF': 'ПРИЛ', 'ADV': 'НАРЕЧ',
            'INFN': 'ИНФ', 'PRTF': 'ПРИЧ', 'PRTS': 'ДЕЕПР',
            'GRND': 'ДЕЕПР', 'NUMR': 'ЧИСЛ', 'ADVB': 'НАРЕЧ',
            'NPRO': 'МЕСТ', 'PRED': 'ПРЕДК', 'PREP': 'ПРЕДЛ',
            'CONJ': 'СОЮЗ', 'PRCL': 'ЧАСТ', 'INTJ': 'МЕЖД'
        }.get(pos, pos)
        
        results.append({
            'word': word,
            'lemma': lemma,
            'pos': pos_russian
        })
        
        if pos_russian:
            pos_counter[pos_russian] += 1
        
        if len(morph.parse(word)) > 1:
            all_parses = morph.parse(word)
            different_pos = set(parse.tag.POS for parse in all_parses)
            
            if len(different_pos) > 1:
                ambiguous_words.append({
                    'word': word,
                    'possible_lemmas': [p.normal_form for p in all_parses],
                    'possible_pos': [pos_russian for p in all_parses]
                })
    
    return results, pos_counter, ambiguous_words

russian_text = """
Машинное обучение становится всё более популярным в современном мире.
Компании используют алгоритмы машинного обучения для анализа данных и принятия решений.
Инженеры по данным обучают модели на больших наборах данных, чтобы повысить их точность.
Эти технологии меняют нашу жизнь, делая её более удобной и эффективной.
"""

results, pos_stats, ambiguous = analyze_russian_text(russian_text)

print("Лемматизация и части речи:")
print("=" * 50)
print(f"{'Слово':<15} {'Лемма':<15} {'Часть речи':<10}")
print("-" * 50)

for item in results:
    print(f"{item['word']:<15} {item['lemma']:<15} {item['pos']:<10}")

print("\n" + "=" * 50)
print("Статистика частей речи:")
print("-" * 50)
for pos, count in pos_stats.most_common():
    print(f"{pos}: {count}")

print("\n" + "=" * 50)
print("Омонимы:")
print("-" * 50)

if ambiguous:
    for amb in ambiguous:
        print(f"Слово: '{amb['word']}'")
        print(f"  Возможные леммы: {amb['possible_lemmas']}")
        print(f"  Возможные части речи: {amb['possible_pos']}")
else:
    print("Неоднозначных слов не найдено")

print("\n" + "=" * 50)
print("Способы разрешения неоднозначности:")
print("-" * 50)
print("1. Использование контекста, соседних слов")
print("2. Применение синтаксического анализа предложения")
print("3. Использование статистических моделей")

AttributeError: module 'inspect' has no attribute 'getargspec'

Упражнение 4: Обработка смешанного русско-английского текста с использованием spaCy

Задание: Разработайте систему для обработки смешанного русско-английского текста с использованием библиотеки spaCy. Ваша программа должна определять язык каждого слова, применять соответствующую модель для лемматизации и выполнять частеречную разметку.

Требования:

* Загрузите модели en_core_web_sm и ru_core_news_sm из spaCy
* Реализуйте алгоритм определения языка для каждого токена (можно использовать множества символов кириллицы и латиницы)
* Обработайте каждый токен соответствующей языковой моделью
* Сформируйте два отдельных набора данных: лемматизированный русский и английский текст
* Определите наиболее часто встречающиеся леммы для каждого языка

In [19]:
import spacy
from collections import Counter

nlp_ru = spacy.load("ru_core_news_sm")
nlp_en = spacy.load("en_core_web_sm")

mixed_text = """
Data science становится ключевой компетенцией в современном IT-мире.
Machine learning инженеры и аналитики данных востребованы на рынке труда.
Компании активно внедряют artificial intelligence решения для оптимизации бизнес-процессов.
Работодатели ищут специалистов, владеющих языком программирования Python и имеющих опыт работы с big data.
"""

def token_language(token_text):
    has_cyrillic = any('а' <= c <= 'я' or 'А' <= c <= 'Я' for c in token_text)
    has_latin = any('a' <= c.lower() <= 'z' for c in token_text)
    
    if has_cyrillic and not has_latin:
        return 'ru'
    elif has_latin and not has_cyrillic:
        return 'en'
    elif has_cyrillic and has_latin:
        cyr_count = sum(1 for c in token_text if 'а' <= c <= 'я' or 'А' <= c <= 'Я')
        lat_count = sum(1 for c in token_text if 'a' <= c.lower() <= 'z')
        return 'ru' if cyr_count >= lat_count else 'en'
    else:
        return 'other'

ru_tokens = []
en_tokens = []

for token in mixed_text.split():
    lang = token_language(token)
    if lang == 'ru':
        ru_tokens.append(token)
    elif lang == 'en':
        en_tokens.append(token)

ru_text = " ".join(ru_tokens)
en_text = " ".join(en_tokens)

doc_ru = nlp_ru(ru_text)
doc_en = nlp_en(en_text)

ru_lemmas = [t.lemma_ for t in doc_ru if t.is_alpha]
en_lemmas = [t.lemma_ for t in doc_en if t.is_alpha]

print("Лемматизированный русский текст:")
for token in doc_ru:
	print(f"{token.text:18} -> {token.lemma_}")

print("\nЛемматизированный английский текст:")
for token in doc_en:
	print(f"{token.text:18} -> {token.lemma_}")

print("\nНаиболее часто встречающиеся леммы в русском:", Counter(ru_lemmas).most_common(8))
print("\nНаиболее часто встречающиеся леммы в английском:", Counter(en_lemmas).most_common(8))



Лемматизированный русский текст:
становится         -> становиться
ключевой           -> ключевой
компетенцией       -> компетенция
в                  -> в
современном        -> современный
IT                 -> it
-                  -> -
мире               -> мир
.                  -> .
инженеры           -> инженер
и                  -> и
аналитики          -> аналитик
данных             -> данных
востребованы       -> востребовать
на                 -> на
рынке              -> рынок
труда              -> труд
.                  -> .
Компании           -> компания
активно            -> активно
внедряют           -> внедрять
решения            -> решение
для                -> для
оптимизации        -> оптимизация
бизнес             -> бизнес
-                  -> -
процессов          -> процессов
.                  -> .
Работодатели       -> работодатель
ищут               -> искать
специалистов       -> специалист
,                  -> ,
владеющих          -> владеть
языком          

Упражнение 5*: Применение библиотеки Natasha для анализа именованных сущностей и лемматизации русского текста

Задание: Используйте библиотеку Natasha для выделения именованных сущностей (NER) и лемматизации русского текста. Создайте функцию, которая принимает текст, выделяет и классифицирует в нём именованные сущности (имена людей, организации, локации и т.д.), а также выполняет лемматизацию всех слов.

Требования:

* Используйте Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger из библиотеки Natasha
* Выделите все именованные сущности в тексте и определите их тип
* Выполните лемматизацию всех слов, учитывая контекст (используя синтаксический анализ)
* Сформируйте структурированный результат, включающий: список всех именованных сущностей с указанием их типа, список всех слов с их леммами, статистику по типам найденных сущностей

In [ ]:
# ваш код

# Текст для проверки

russian_ner_text = """
Президент России Владимир Путин провел совещание с правительством в Москве.
Компания Яндекс представила новую технологию для анализа данных.
Сотрудники Московского государственного университета и Института проблем искусственного интеллекта РАН
разработали алгоритм для обработки естественного языка.
Исследователи из Санкт-Петербурга и их коллеги из Казани опубликовали результаты исследований
в журнале "Компьютерная лингвистика и интеллектуальные технологии".
"""